In [1]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_breast_cancer, load_wine
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_validate
from sklearn.model_selection import ParameterGrid, GridSearchCV, RandomizedSearchCV
from IPython.display import display
pd.set_option('max_rows', 5)
pd.set_option('max_columns', 9)

# バリデーション

## バリデーション (validation)
---
ハイパーパラメータ調整のために、学習に使用していないデータを用いてモデルを評価すること。

機械学習ではモデルのパラメータはデータから自動的に学習するものとユーザーが設定するもの (クラス引数に与える値) がある。後者を特にハイパーパラメータと呼ぶ。

<table class="border text-center">
    <tr class="background-dark">
        <th>種類</th>
        <th>名称</th>
        <th>具体例</th>
    </tr>
    <tr class="background-bright">
        <td>データから自動的に学習するもの</td>
        <td>パラメータ</td>
        <td>$w$ (ウェイト) や $b$ (バイアス)</td>
    </tr>
    <tr class="background-bright">
        <td>ユーザーが設定するもの</td>
        <td>ハイパーパラメータ</td>
        <td>L1 ・ L2正則化の $\alpha$</td>
    </tr>
</table>

ハイパーパラメータの値をチューニング (調整) することで精度を向上できる。

## バリデーションの必要性
---
テストデータでの精度向上を目指してチューニングすると、ハイパーパラメータの値にテストデータの情報が盛り込まれてしまう。  
= 手動でテストデータの情報をモデルに与え、**テストデータに対して過学習**を起こし、本当に未知のデータに対する汎化性能が測定できなくなる。

 1. あるハイパーパラメータの値 $( \theta _{1})$ を使ってトレーニングデータ $( x_{train} ,y_{train})$ で学習
 1. テストデータ $( x_{test} ,y_{test})$ での精度 $( L_{\theta _{1}})$ を見る
 1. 別のハイパーパラメータの値 $( \theta _{2} ,\theta _{3} ,\cdots )$ を試す
 1. テストデータで精度がよかったハイパーパラメータ $( \theta _{best})$ を採用
 1. 採用したハイパーパラメータ $( \theta _{best})$ はテストデータで精度が出るように調整されたものなので、実際に運用してみると期待した精度が出ない

そこで、ハイパーパラメータのチューニングには、データセットをトレーニングデータ・バリデーションデータ・テストデータに分割し、

 - トレーニングデータで学習 → バリデーションデータでハイパーパラメータをチューニング → テストデータで汎化能力確認

<table class="text-center">
    <tr>
        <th colspan="5">データセット</th>
    </tr>
    <tr class="border-bottom">
        <th>トレーニングデータ</th>
        <th></th>
        <th>バリデーションデータ</th>
        <th></th>
        <th>テストデータ</th>
    </tr>
    <tr>
        <td>学習</td>
        <td>→</td>
        <td>ハイパーパラメータのチューニング</td>
        <td>→</td>
        <td>汎化性能の確認</td>
    </tr>
</table>

という手順を踏む。

バリデーションデータを使ってハイパーパラメータを決定した後は、そのハイパーパラメータとデータセット全体 (トレーニングデータ+バリデーションデータ) を使ってパラメータを学習させる。

 1. あるハイパーパラメータの値 $( \theta _{1})$ を使ってトレーニングデータ $( x_{train} ,y_{train})$ で学習
 1. バリデーションデータ $( x_{valid} ,y_{valid})$ での精度 $( L_{\theta _{1}})$ を見る
 1. 別のハイパーパラメータの値 $( \theta _{2} ,\theta _{3} ,\cdots )$ を試す
 1. バリデーションデータで精度がよかったハイパーパラメータ $( \theta _{best})$ を採用
 1. 採用したハイパーパラメータ $( \theta_{best})$ を使って学習用の全データ $( x_{train}+x_{valid} ,y_{train}+y_{valid})$ で学習
 1. テストデータ $( x_{test} ,y_{test})$ で汎化性能を確認

## バリデーションの種類

### ホールドアウト法
---
テストと同様にデータセットの一部からバリデーション用のデータを取り出し、トレーニングデータで学習、バリデーションデータでハイパーパラメーターの性能を評価する方法。

トレーニングデータ・バリデーションデータ・テストデータの割合は、 $6:2:2$ や $8:1:1$ などバリデーションデータとテストデータの割合を同程度にすることが多い。

#### Pythonでのホールドアウト法の実行方法
---
テストデータと同様に`sklearn.model_selection.train_test_split`を使用する。

### 交差検証法 (cross-validation)
---
訓練データを $k$ 個に分割し、1個をバリデーションデータ・残りを訓練データとして利用し性能評価、それを全ての組で行って、平均性能をそのハイパーパラメーターの性能とする。

<table class="border text-center">
    <tr class="background-dark">
        <th colspan="6">パラメータ $\theta _{1}$ の学習</th>
    </tr>
    <tr class="background-dark">
        <td></td>
        <td>データセット1</td>
        <td>データセット2</td>
        <td>…</td>
        <td>データセットk</td>
        <td>平均</td>
    </tr>
    <tr class="background-bright">
        <td>1回目</td>
        <td>バリデーションデータ</td>
        <td>トレーニングデータ</td>
        <td>…</td>
        <td>トレーニングデータ</td>
        <td rowspan="4">$\theta _{1}$ の性能</td>
    </tr>
    <tr class="background-bright">
        <td>2回目</td>
        <td>トレーニングデータ</td>
        <td>バリデーションデータ</td>
        <td>…</td>
        <td>トレーニングデータ</td>
    </tr>
    <tr class="background-bright">
        <td colspan="5">$\vdots$</td>
    </tr>
    <tr class="background-bright">
        <td>k回目</td>
        <td>トレーニングデータ</td>
        <td>トレーニングデータ</td>
        <td>…</td>
        <td>バリデーションデータ</td>
    </tr>
    <tr class="border-none background-default">
        <td colspan="6"></td>
    </tr>
    <tr class="background-dark">
        <th colspan="6">パラメータ $\theta _{2}$ の学習</th>
    </tr>
    <tr class="background-dark">
        <td></td>
        <td>データセット1</td>
        <td>データセット2</td>
        <td>…</td>
        <td>データセットk</td>
        <td>平均</td>
    </tr>
    <tr class="background-bright">
        <td>1回目</td>
        <td>バリデーションデータ</td>
        <td>トレーニングデータ</td>
        <td>…</td>
        <td>トレーニングデータ</td>
        <td rowspan="4">$\theta _{2}$ の性能</td>
    </tr>
    <tr class="background-bright">
        <td>2回目</td>
        <td>トレーニングデータ</td>
        <td>バリデーションデータ</td>
        <td>…</td>
        <td>トレーニングデータ</td>
    </tr>
    <tr class="background-bright">
        <td colspan="5">$\vdots$</td>
    </tr>
    <tr class="background-bright">
        <td>k回目</td>
        <td>トレーニングデータ</td>
        <td>トレーニングデータ</td>
        <td>…</td>
        <td>バリデーションデータ</td>
    </tr>
    <tr class="border-none background-default">
        <td colspan="6">$\vdots$</td>
    </tr>
</table>

 - トレーニングデータとバリデーションデータの分け方による誤差に強い
 - 各ハイパーパラメーターごとに $k$ 回学習を行うので、実行にかかる時間が長くなる

特に分類問題で完全にランダムにデータを分割すると、各グループでクラスの偏りが発生する可能性があるので、元データのクラス比率を維持しながら分割する層化 k 分割交差検証 (stratified k-fold cross-validation) を用いることも多い。

極端にデータが少ない場合にはバリデーションデータとして 1 サンプルのみを使用する LOO (Leave One Out) も使われることがある。

#### クロスバリデーションとテスト
---
クロスバリデーションを実施する際もテストを行うのが望ましいが、計算コストの高いクロスバリデーションは**データが少ない場合に使われることが多く**、テストデータを十分に確保できない。また、 1 つのハイパーパラメーターに対して複数のデータ分割で評価しているので、ホールドアウト法より過学習を起こしにくい。  
そのため、クロスバリデーションを実施する場合にはテストは行わないことも多い。

#### Pythonでのクロスバリデーションの実行方法
---
`sklearn.model_selection.cross_validate`や各モデル名の後に CV のついたもの (`RidgeCV`など) を使用する。

In [2]:
loader = load_wine()
wine = pd.DataFrame(np.column_stack([loader.data, loader.target]),
                    columns=list(loader.feature_names) + ['target'])
print('wine')
display(wine)

wine


,alcohol,malic_acid,ash,alcalinity_of_ash,...,hue,od280/od315_of_diluted_wines,proline,target
0,14.23,1.71,2.43,15.6,...,1.04,3.92,1065.0,0.0
1,13.20,1.78,2.14,11.2,...,1.05,3.40,1050.0,0.0
...,...,...,...,...,...,...,...,...,...
176,13.17,2.59,2.37,20.0,...,0.60,1.62,840.0,2.0
177,14.13,4.10,2.74,24.5,...,0.61,1.60,560.0,2.0


In [3]:
help(cross_validate)

Help on function cross_validate in module sklearn.model_selection._validation:

cross_validate(estimator, X, y=None, *, groups=None, scoring=None, cv=None, n_jobs=None, verbose=0, fit_params=None, pre_dispatch='2*n_jobs', return_train_score=False, return_estimator=False, error_score=nan)
    Evaluate metric(s) by cross-validation and also record fit/score times.
    
    Read more in the :ref:`User Guide <multimetric_cross_validation>`.
    
    Parameters
    ----------
    estimator : estimator object implementing 'fit'
        The object to use to fit the data.
    
    X : array-like of shape (n_samples, n_features)
        The data to fit. Can be for example a list, or an array.
    
    y : array-like of shape (n_samples,) or (n_samples, n_outputs),             default=None
        The target variable to try to predict in the case of
        supervised learning.
    
    groups : array-like of shape (n_samples,), default=None
        Group labels for the samples used while splitt

In [4]:
x = wine.iloc[:, :-1]
y = wine.iloc[:, -1]
model = DecisionTreeClassifier()
cross_validate(model, x, y, cv=5, n_jobs=-1)

{'fit_time': array([0.00844717, 0.09507132, 0.00844002, 0.01233053, 0.00655556]),
 'score_time': array([0.00326467, 0.0130744 , 0.00452209, 0.00895691, 0.00270534]),
 'test_score': array([0.94444444, 0.83333333, 0.88888889, 0.91428571, 0.82857143])}

## ハイパーパラメーターサーチ
---
様々なハイパーパラメーターを試して、精度のいいものを探すこと。

### グリッドサーチ
---
与えられたハイパーパラメーターの値候補の全組み合わせを試して、最も精度のよい組み合わせを探す方法。

#### Pythonでのグリッドサーチの実行方法
---
`sklearn.model_selection.GridSearchCV`を使用する。`sklearn.model_selection.ParameterGrid`で探索するハイパーパラメーターの全組み合わせを作成しておく必要がある。

In [5]:
help(ParameterGrid)

Help on class ParameterGrid in module sklearn.model_selection._search:

class ParameterGrid(builtins.object)
 |  ParameterGrid(param_grid)
 |  
 |  Grid of parameters with a discrete number of values for each.
 |  
 |  Can be used to iterate over parameter value combinations with the
 |  Python built-in function iter.
 |  
 |  Read more in the :ref:`User Guide <grid_search>`.
 |  
 |  Parameters
 |  ----------
 |  param_grid : dict of str to sequence, or sequence of such
 |      The parameter grid to explore, as a dictionary mapping estimator
 |      parameters to sequences of allowed values.
 |  
 |      An empty dict signifies default parameters.
 |  
 |      A sequence of dicts signifies a sequence of grids to search, and is
 |      useful to avoid exploring parameter combinations that make no sense
 |      or have no effect. See the examples below.
 |  
 |  Examples
 |  --------
 |  >>> from sklearn.model_selection import ParameterGrid
 |  >>> param_grid = {'a': [1, 2], 'b': [True,

In [6]:
params = dict(criterion=['gini', 'entropy'], max_depth=[3, 4, 5])
grid = ParameterGrid(params)
list(grid)

[{'criterion': 'gini', 'max_depth': 3},
 {'criterion': 'gini', 'max_depth': 4},
 {'criterion': 'gini', 'max_depth': 5},
 {'criterion': 'entropy', 'max_depth': 3},
 {'criterion': 'entropy', 'max_depth': 4},
 {'criterion': 'entropy', 'max_depth': 5}]

In [7]:
help(GridSearchCV)

Help on class GridSearchCV in module sklearn.model_selection._search:

class GridSearchCV(BaseSearchCV)
 |  GridSearchCV(estimator, param_grid, *, scoring=None, n_jobs=None, iid='deprecated', refit=True, cv=None, verbose=0, pre_dispatch='2*n_jobs', error_score=nan, return_train_score=False)
 |  
 |  Exhaustive search over specified parameter values for an estimator.
 |  
 |  Important members are fit, predict.
 |  
 |  GridSearchCV implements a "fit" and a "score" method.
 |  It also implements "predict", "predict_proba", "decision_function",
 |  "transform" and "inverse_transform" if they are implemented in the
 |  estimator used.
 |  
 |  The parameters of the estimator used to apply these methods are optimized
 |  by cross-validated grid-search over a parameter grid.
 |  
 |  Read more in the :ref:`User Guide <grid_search>`.
 |  
 |  Parameters
 |  ----------
 |  estimator : estimator object.
 |      This is assumed to implement the scikit-learn estimator interface.
 |      Either e

In [8]:
gs = GridSearchCV(model, params, n_jobs=-1, cv=5)
gs.fit(x, y)
gs.cv_results_

{'mean_fit_time': array([0.0364994 , 0.01473827, 0.0245657 , 0.01613584, 0.0151865 ,
        0.01307125]),
 'std_fit_time': array([0.01408598, 0.0059776 , 0.01026655, 0.0088567 , 0.00873395,
        0.00330426]),
 'mean_score_time': array([0.01353846, 0.01218252, 0.01154552, 0.00857816, 0.00781469,
        0.00542741]),
 'std_score_time': array([0.00682476, 0.01307534, 0.0058919 , 0.00519834, 0.00255785,
        0.000248  ]),
 'param_criterion': masked_array(data=['gini', 'gini', 'gini', 'entropy', 'entropy',
                    'entropy'],
              mask=[False, False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'param_max_depth': masked_array(data=[3, 4, 5, 3, 4, 5],
              mask=[False, False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'criterion': 'gini', 'max_depth': 3},
  {'criterion': 'gini', 'max_depth': 4},
  {'criterion': 'gini', 'max_depth': 5},
  {'criterion': 'entropy', 'max_

In [9]:
gs.best_params_

{'criterion': 'gini', 'max_depth': 4}

### ランダムサーチ
---
与えられたハイパーパラメーターの値候補からランダムに組み合わせて、精度のよい組み合わせを探す方法。  
複数のハイパーパラメーターのうち、精度に大きく影響するものは少数であったり、どのハイパーパラメーターをどのくらいの範囲で探索するのが効率的かわからなかったりするので、グリッドサーチを実施する前に絞り込みに利用したりする。

#### Pythonでのランダムサーチの実行方法
---
`sklearn.model_selection.RandomizedSearchCV`を使用する。

In [10]:
help(RandomizedSearchCV)

Help on class RandomizedSearchCV in module sklearn.model_selection._search:

class RandomizedSearchCV(BaseSearchCV)
 |  RandomizedSearchCV(estimator, param_distributions, *, n_iter=10, scoring=None, n_jobs=None, iid='deprecated', refit=True, cv=None, verbose=0, pre_dispatch='2*n_jobs', random_state=None, error_score=nan, return_train_score=False)
 |  
 |  Randomized search on hyper parameters.
 |  
 |  RandomizedSearchCV implements a "fit" and a "score" method.
 |  It also implements "predict", "predict_proba", "decision_function",
 |  "transform" and "inverse_transform" if they are implemented in the
 |  estimator used.
 |  
 |  The parameters of the estimator used to apply these methods are optimized
 |  by cross-validated search over parameter settings.
 |  
 |  In contrast to GridSearchCV, not all parameter values are tried out, but
 |  rather a fixed number of parameter settings is sampled from the specified
 |  distributions. The number of parameter settings that are tried is
 | 

In [11]:
rs = RandomizedSearchCV(model,
                        params,
                        n_iter=3,
                        n_jobs=-1,
                        cv=5,
                        random_state=1234)
rs.fit(x, y)
rs.cv_results_

{'mean_fit_time': array([0.01502481, 0.02238603, 0.00731196]),
 'std_fit_time': array([0.01079678, 0.01055039, 0.00118628]),
 'mean_score_time': array([0.00596137, 0.00772924, 0.00674858]),
 'std_score_time': array([0.00326713, 0.00433882, 0.00370852]),
 'param_max_depth': masked_array(data=[5, 4, 5],
              mask=[False, False, False],
        fill_value='?',
             dtype=object),
 'param_criterion': masked_array(data=['gini', 'gini', 'entropy'],
              mask=[False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'max_depth': 5, 'criterion': 'gini'},
  {'max_depth': 4, 'criterion': 'gini'},
  {'max_depth': 5, 'criterion': 'entropy'}],
 'split0_test_score': array([0.88888889, 0.91666667, 0.91666667]),
 'split1_test_score': array([0.77777778, 0.77777778, 0.83333333]),
 'split2_test_score': array([0.88888889, 0.94444444, 0.94444444]),
 'split3_test_score': array([0.91428571, 0.91428571, 0.97142857]),
 'split4_test_score': array([0.857142

In [12]:
rs.best_params_

{'max_depth': 5, 'criterion': 'entropy'}

## 推薦図書
---
- [Python 機械学習プログラミング 達人データサイエンティストによる理論と実践](https://www.amazon.co.jp/Python-%E6%A9%9F%E6%A2%B0%E5%AD%A6%E7%BF%92%E3%83%97%E3%83%AD%E3%82%B0%E3%83%A9%E3%83%9F%E3%83%B3%E3%82%B0-%E9%81%94%E4%BA%BA%E3%83%87%E3%83%BC%E3%82%BF%E3%82%B5%E3%82%A4%E3%82%A8%E3%83%B3%E3%83%86%E3%82%A3%E3%82%B9%E3%83%88%E3%81%AB%E3%82%88%E3%82%8B%E7%90%86%E8%AB%96%E3%81%A8%E5%AE%9F%E8%B7%B5-impress-gear/dp/4295003379/)
- [Kaggleで勝つデータ分析の技術](https://www.amazon.co.jp/Kaggle%E3%81%A7%E5%8B%9D%E3%81%A4%E3%83%87%E3%83%BC%E3%82%BF%E5%88%86%E6%9E%90%E3%81%AE%E6%8A%80%E8%A1%93-%E9%96%80%E8%84%87-%E5%A4%A7%E8%BC%94/dp/4297108437/)